
# Areas of Malaysia : GeoCoding for Informed Decision Making 
Accessing and Analyzing GeoSpacial Data.
***
#### Table of Content 
1. Problem Statment
    - Objectives
2. Data 
    - Identifying the Required Data
    - Sourcing the Required Data 
    - Cleaning and Correlating the Data 
3. Methadology:
    - Structuring and organizing the Areas 
    - Retrieving their Veniues 
    - Identifying Common Areas 
    - Klustering to Identifying Unique Areas to study
4. Results
    - Disucssing Overall Outcome 
    - Discussing Clustering Outcome
    - Discussing Customer living priorities
    - Discusing Business Location Priorties 
5. Conclusion 
***


## Problem Statment 
With the financial, economical and geographic turmoil of 2020 - the need for depending on closer vicinity  businesses has become highlighted more than ever. Identifying the correct place to live and start a business has become very relevant  and closely tied to the existence of venues and business within the radius of the actual Physical locations identified. Is there a data Drive approch to select and identify such appropriate arease.


### Objectives: 
On the 16th of March 2020, and as a result of the covid 19 case numbers - the Malaysian Government, following suite to what many other sister countries instructed - put to place a Movement Restriction order - restricting the movement of its citizens - allowing for only travel within a specific radius. What followed was a year of Standard operating procedures and new normals - a shift in lifestyle that affected both business and family lifestyles. 

With the financial, economical and geographic turmoil of 2020 - the need for depending on closer vicinity  businesses has become highlighted more than ever. Identifying the correct place to live and start a business has become very relevant  and closely tied to the existence of venues and business within the radius of the actual Physical locations identified.

It became very important to scout out and be informed of the facilities available in a certain area - ensuring that both lifestyle priorities are met. 

### Problem
For our problem statement - we are tasked with identifying a data driven approach to decision making to help us cope with the new normal. Through the use of Geo-Analysis , we aim to Recommend.

The Best Area to Live:
      Selecting the best area to live is often a subjective decision as it must align with the personal priorities of the Homeowner/Renter - but to take a data driven approach to the question - we are aiming to create recommendations that can best fit a specific lifestyle - with respect to the availability of the facilities required for that lifestyle within vicinity. For our Case study - we are interested to find an area suitable for an athletic lifestyle interested in quite night drinks and diversified cuisine. 
      
      
The Best area to start a Restaurant 
    Starting a business during a pandemic is a very risky task - as such and in efforts to reduce the assigned risk - we aim to identify the best area to start a business. Specifically we aim to identify the best area to start an arab/Middle Eastern Restaurant with respect to frequency of venues within radius. 
For the Purposes of this report - the Locations Referenced will be from Malaysia Specifically the Kuala Lumpur, Johor baru Areas. 




In [1]:
#identifying the Correlating Libraries

import pandas as pd, numpy as np #Libraries Used to Store, manipulated data
import requests #Used to Pull information from the internte
import json #Used to access and interpret location data
import time
import folium   #for creating maps
from pandas.io.json import json_normalize #converting json to DataFrame
from geopy.geocoders import Nominatim  #converting address to cordinates
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

## Data
For our objectives - we must first study the data applicable to our situation. to insure we are able to compare and study the areas in KL and JB - we must first identify how we can :
1. Break up the Geographic area to Analyze
2. Identify Source of PostalCode Latitude and Longitude Data
3. Identify Source of Venues .


Postal Code data #https://www.aggdata.com/free/malaysia-postal-codes


#### Data:  Break up the Geographic area to Analyze
To help our Clients to identify how to break down geographic areas to compare - we identift the division of areas by postol codes - first specifiting to only the postol Codes within the KL and JB Area
To do so we use 2 Datasets holding the different areas within the KL and JB districs and their postol Codes
- https://raw.githubusercontent.com/zaephaer/CapstoneProject/master/JB_disrict.csv
- https://raw.githubusercontent.com/zaephaer/CapstoneProject/master/KL_disrict.csv


In [2]:
#DataSet1
JB_district = pd.read_csv('https://raw.githubusercontent.com/zaephaer/CapstoneProject/master/JB_disrict.csv')
JB_district.head()

,Postcode,District,Area
0,80000,Johor Bahru,Johor Bahru
1,81100,Johor Bahru,Bandar Dato' Onn
2,80200,Johor Bahru,Danga Bay
3,81100,Johor Bahru,Johor Jaya
4,81100,Johor Bahru,Desa Jaya


In [3]:
#DataSet2
KL_district = pd.read_csv('https://raw.githubusercontent.com/zaephaer/CapstoneProject/master/KL_disrict.csv')


In [4]:
#DataSet3
postalcodedata = pd.read_csv('my_postal_codes.csv')

In [5]:

#examine data
print('Kuala Lumpur dataframe has {} district and {} areas.'.format(
        len(KL_district['District'].unique()),
        KL_district.shape[0]
    )
)

#grouping data to find District with highest number of area
KL_district.groupby('District').count()

Kuala Lumpur dataframe has 11 district and 66 areas.


,Postcode,Area
District,,
Bandar Tun Razak,6,6
Batu,2,2
Bukit Bintang,11,11
Cheras,9,9
Kepong,2,2
Lembah Pantai,6,6
Segambut,11,11
Seputeh,8,8
Setiawangsa,3,3


In [6]:

#examine data
print('Johor Bahru dataframe has {} district and {} areas.'.format(
        len(JB_district['District'].unique()),
        JB_district.shape[0]
    )
)

#grouping data to find District with highest number of area
JB_district.groupby('District').count()

Johor Bahru dataframe has 3 district and 22 areas.


,Postcode,Area
District,,
Iskandar Puteri,8,8
Johor Bahru,10,10
Pasir Gudang,4,4


#### Data:  Identify Source of PostalCode Latitude and Longitude Data
To Correlate and reverify the Postol Code - we retrieve a dataset from the Malaysian Goverment Website - correlating the Postal Codes to their Specific Latitiude and Longitude Locations . We Then Comibe them to prepare two indivisual datasets prepared for processing 

In [7]:
postalcodedata.head()

,Postal Code,Place Name,State,State Abbreviation,Latitude,Longitude
0,79000.0,Nusajaya,Johor,JHR,1.5014,103.6439
1,79100.0,Nusajaya,Johor,JHR,1.4835,103.6341
2,79150.0,Nusajaya,Johor,JHR,1.4968,103.6524
3,79200.0,Nusajaya,Johor,JHR,1.4836,103.6364
4,79250.0,Nusajaya,Johor,JHR,1.5034,103.6522


In [8]:
postalcodedata.dtypes

Postal Code           float64
Place Name             object
State                  object
State Abbreviation     object
Latitude              float64
Longitude             float64
dtype: object

In [9]:
#examine data
print('The Postal Code data set contains information about {} Places and {} States.'.format(
        len(postalcodedata['Place Name'].unique()),
        len(postalcodedata['State'].unique())
    )
)

#grouping data to find District with highest number of area
postalcodedata.groupby('State').count()

The Postal Code data set contains information about 433 Places and 17 States.


,Postal Code,Place Name,State Abbreviation,Latitude,Longitude
State,,,,,
Johor,154,154,154,154,154
Kedah,145,145,145,145,145
Kelantan,131,131,131,131,131
Kuala Lumpur,278,278,278,278,278
Labuan,25,25,25,25,25
Melaka,116,116,116,116,116
Negeri Sembilan,154,154,154,154,154
Pahang,197,197,197,197,197
Perak,228,228,228,228,228


In [10]:
postalcodedata.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [11]:
JB_merged = pd.merge(JB_district,postalcodedata, on = 'Postcode')

In [12]:
KL_Merged =  pd.merge(KL_district,postalcodedata, on = 'Postcode')

In [13]:
KL_Merged.head()

,Postcode,District,Area,Place Name,State,State Abbreviation,Latitude,Longitude
0,52100,Kepong,Jinjang,Kuala Lumpur,Kuala Lumpur,KUL,3.1833,101.6584
1,52100,Kepong,Taman Bukit Maluri,Kuala Lumpur,Kuala Lumpur,KUL,3.1833,101.6584
2,51200,Segambut,Bandar Menjalara,Kuala Lumpur,Kuala Lumpur,KUL,3.1876,101.6720
3,51200,Segambut,Bukit Kiara,Kuala Lumpur,Kuala Lumpur,KUL,3.1876,101.6720
4,51200,Segambut,Bukit Tunku,Kuala Lumpur,Kuala Lumpur,KUL,3.1876,101.6720


In [14]:
UsedJB = JB_merged[["Postcode","District","Area","Latitude","Longitude"]]
UsedKL = KL_Merged[["Postcode","District","Area","Latitude","Longitude"]]

In [15]:
UsedJB.head()

,Postcode,District,Area,Latitude,Longitude
0,80000,Johor Bahru,Johor Bahru,1.5035,103.7405
1,81100,Johor Bahru,Bandar Dato' Onn,1.5495,103.7080
2,81100,Johor Bahru,Johor Jaya,1.5495,103.7080
3,81100,Johor Bahru,Desa Jaya,1.5495,103.7080
4,81100,Johor Bahru,Ehsan Jaya,1.5495,103.7080


In [16]:
#Having retrieved the data we are able to retrieve the data from the 
#Processing the KL data:
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="On_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumbur are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumbur are 3.1516964, 101.6942371.


In [17]:
UsedKL.head()

,Postcode,District,Area,Latitude,Longitude
0,52100,Kepong,Jinjang,3.1833,101.6584
1,52100,Kepong,Taman Bukit Maluri,3.1833,101.6584
2,51200,Segambut,Bandar Menjalara,3.1876,101.6720
3,51200,Segambut,Bukit Kiara,3.1876,101.6720
4,51200,Segambut,Bukit Tunku,3.1876,101.6720


In [18]:


# create map of Kuala Lumbur using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(UsedKL['Latitude'], UsedKL['Longitude'], UsedKL['Area'], UsedKL['District']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kl)  
    
map_kl

In [19]:
#Having retrieved the data we are able to retrieve the data from the 
#Processing the KL data:
address2 = 'Johor Bahru, Malaysia'

geolocator = Nominatim(user_agent="On_explorer")
location = geolocator.geocode(address2)
latitude2 = location.latitude
longitude2 = location.longitude
print('The geograpical coordinate of Kuala Lumbur are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumbur are 3.1516964, 101.6942371.


In [20]:


# create map of Kuala Lumbur using latitude and longitude values
map_jb = folium.Map(location=[latitude2, longitude2], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(UsedJB['Latitude'], UsedJB['Longitude'], UsedJB['Area'], UsedJB['District']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_jb)  
    
map_jb


#### Identify Source of Venues Data To Correlate to Locational Data 

Using the Foursquare api - we are able to use the geographic location of each postal code to retrieve the venues in each area and thus prepare for our analysis. The Foursquare Places API provides location based experiences with diverse information about venues, users, photos, and check-ins. The API supports real time access to places, Snap-to-Place that assigns users to specific locations, and Geo-tag 


In [21]:
CLIENT_ID = '5LBINWUFIF2VBBDKH4JKUXFWW1Q4C11X1EU33ZLMAW3SVYAO' # Our Foursquare ID
CLIENT_SECRET = 'M0EPXTMWM3RVUDUXPDSOO30LATTINQIUSNCWZV3G3EQJ43KL' # Our Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
#We Retrieved the Venues
KL_Venues = getNearbyVenues(names=UsedKL['Area'],
                                   latitudes=UsedKL['Latitude'],
                                   longitudes=UsedKL['Longitude']
                                  )

Jinjang
Taman Bukit Maluri
Bandar Menjalara
Bukit Kiara
Bukit Tunku
Damansara
Damansara Town Centre
Jalan Duta
Kampung Kasipillay
Kampung Sungai Penchala
Mont Kiara
Sri Hartamas
Taman Tun Dr Ismail
Bandar Baru Sentul
Sentul
Setapak
Taman Melati
Taman P. Ramlee
Kampung Datuk Keramat
Semarak
Desa Tun Hussein Onn
Ampang
Jalan Cochrane
Maluri
Kampung Baru
Kampung Pandan
KL Sentral
Bukit Nanas
Bukit Petaling
Chow Kit
Dang Wangi
Kampung Baru
KL City Centre
Medan Tuanku
Pudu
Salak South
Tun Razak Exchange
Bangsar
Bangsar South
Federal Hill
Kg Abdullah Hukum
Mid Valley City
Pantai Dalam
Bandar Malaysia
Bukit Jalil
Bukit Petaling
Happy Garden
Kuchai Lama
Sri Petaling
Taman Desa
Taman OUG
Alam Damai
Maluri
Pudu
Shamelin Perkasa
Taman Cheras Hartamas
Taman Connaught
Taman Midah
Taman Len Seng
Taman Taynton View
Bandar Sri Permaisuri
Bandar Tasik Selatan
Desa Petaling
Kampung Malaysia
Lake Fields
Sungai Besi


In [24]:
print(KL_Venues.shape)
KL_Venues.head()

(5002, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jinjang,3.1833,101.6584,Ikan bakar segambut,3.184052,101.659342,Malay Restaurant
1,Jinjang,3.1833,101.6584,Menara Duta Swimming Pool,3.180680,101.657973,Pool
2,Jinjang,3.1833,101.6584,Kak Ton D'Condo,3.184135,101.659502,Malay Restaurant
3,Jinjang,3.1833,101.6584,Thai Avenue Restaurant,3.186287,101.656285,Thai Restaurant
4,Jinjang,3.1833,101.6584,Gerai Adam,3.184156,101.659937,Halal Restaurant


In [25]:
KL_Venues.groupby('Area').count()


,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
Alam Damai,51,51,51,51,51,51
Ampang,96,96,96,96,96,96
Bandar Baru Sentul,66,66,66,66,66,66
Bandar Malaysia,86,86,86,86,86,86
Bandar Menjalara,55,55,55,55,55,55
Bandar Sri Permaisuri,71,71,71,71,71,71
Bandar Tasik Selatan,71,71,71,71,71,71
Bangsar,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [26]:
print('There are {} uniques categories.'.format(len(KL_Venues['Venue Category'].unique())))

There are 164 uniques categories.


In [27]:
JB_venues =  getNearbyVenues(names=UsedJB['Area'],
                                   latitudes=UsedJB['Latitude'],
                                   longitudes=UsedJB['Longitude']
                                  )

Johor Bahru
Bandar Dato' Onn
Johor Jaya
Desa Jaya
Ehsan Jaya
Tebrau
Danga Bay
Larkin
Tampoi
Ulu Tiram
Bukit Indah
Horizon Hills
Gelang Patah
Tanjung Kupang
Medini Iskandar
Skudai
Ulu Choh
Bandar Seri Alam
Masai
Permas Jaya
 Tanjung Langsat


In [28]:
print(JB_venues.shape)
JB_venues.head()

(709, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Johor Bahru,1.5035,103.7405,Goreng Pisang Cheese,1.502802,103.741226,Snack Place
1,Johor Bahru,1.5035,103.7405,Kacang Pool Haji,1.497366,103.743195,Middle Eastern Restaurant
2,Johor Bahru,1.5035,103.7405,Haji Wahid Mee Rebus Pasar Larkin,1.496156,103.743685,Malay Restaurant
3,Johor Bahru,1.5035,103.7405,Selera Johor,1.497339,103.743167,Food Court
4,Johor Bahru,1.5035,103.7405,Putu Piring Larkin,1.501877,103.743447,Snack Place


In [29]:
JB_venues.groupby('Area').count()

,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
Tanjung Langsat,34,34,34,34,34,34
Bandar Dato' Onn,44,44,44,44,44,44
Bandar Seri Alam,34,34,34,34,34,34
Bukit Indah,7,7,7,7,7,7
Danga Bay,78,78,78,78,78,78
Desa Jaya,44,44,44,44,44,44
Ehsan Jaya,44,44,44,44,44,44
Gelang Patah,38,38,38,38,38,38
Horizon Hills,7,7,7,7,7,7


In [30]:
print('There are {} uniques categories.'.format(len(JB_venues['Venue Category'].unique())))

There are 103 uniques categories.


## Methadology 
Having Retrieved the Venues associated with each area - we begin to analyze and understand the venues we are operating. 
1. Encoding Values and Vatagories 
Through encoding the catagories of each venue - we are able to attach numerical values into  catagories and thus are able to compare venues across different catagories.
2. Identify Frequency Disturbution for Each Area
After encoding our values - we can plot a Frequency disturbution to allow us identify the top 5 venue catagories for each area with respect to its 
3. Identify Most Common catagorie for each Area

4. Cluster to Identify Correlated Venues across Areas

In [31]:
Kl_onehot = pd.get_dummies(KL_Venues[['Venue Category']], prefix="", prefix_sep="")
Kl_onehot['Area'] = KL_Venues['Area'] 
fixed_columns = [Kl_onehot.columns[-1]] + list(Kl_onehot.columns[:-1])
Kl_onehot = Kl_onehot[fixed_columns]

Kl_onehot.head()

,Area,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,...,Track Stadium,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Wine Bar,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo Exhibit
0,Jinjang,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Jinjang,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Jinjang,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Jinjang,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Jinjang,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
KL_grouped = Kl_onehot.groupby('Area').mean().reset_index()

In [33]:
num_top_venues = 5

for hood in KL_grouped['Area']:
    print("----"+hood+"----")
    temp = KL_grouped[KL_grouped['Area'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alam Damai----
                venue  freq
0  Chinese Restaurant  0.18
1    Asian Restaurant  0.14
2        Noodle House  0.08
3          Restaurant  0.06
4    Malay Restaurant  0.06


----Ampang----
                   venue  freq
0       Malay Restaurant  0.28
1  Indonesian Restaurant  0.05
2       Asian Restaurant  0.05
3                   Café  0.04
4             Restaurant  0.03


----Bandar Baru Sentul----
              venue  freq
0  Malay Restaurant  0.20
1  Asian Restaurant  0.08
2              Park  0.05
3   Thai Restaurant  0.05
4        Restaurant  0.03


----Bandar Malaysia----
               venue  freq
0  Indian Restaurant  0.12
1        Coffee Shop  0.07
2               Café  0.07
3  Convenience Store  0.07
4              Hotel  0.05


----Bandar Menjalara----
                  venue  freq
0    Chinese Restaurant  0.18
1                  Café  0.09
2      Asian Restaurant  0.07
3          Burger Joint  0.05
4  Gym / Fitness Center  0.05


----Bandar Sri Permaisuri---

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
#Based on the frequency distubution we can identenfy and correlate a list of the most common venues for each topic
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Area_venues_sorted = pd.DataFrame(columns=columns)
Area_venues_sorted['Area'] = KL_grouped['Area']

for ind in np.arange(KL_grouped.shape[0]):
    Area_venues_sorted.iloc[ind, 1:] = return_most_common_venues(KL_grouped.iloc[ind, :], num_top_venues)

Area_venues_sorted.head()

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alam Damai,Chinese Restaurant,Asian Restaurant,Noodle House,Malay Restaurant,Restaurant,Convenience Store,Steakhouse,Food Truck,Japanese Restaurant,Dessert Shop
1,Ampang,Malay Restaurant,Asian Restaurant,Indonesian Restaurant,Café,Chinese Restaurant,Restaurant,Indian Restaurant,Hotel,Steakhouse,Market
2,Bandar Baru Sentul,Malay Restaurant,Asian Restaurant,Thai Restaurant,Park,Restaurant,Arcade,Indian Restaurant,Fast Food Restaurant,Burger Joint,Convenience Store
3,Bandar Malaysia,Indian Restaurant,Coffee Shop,Convenience Store,Café,Malay Restaurant,Hotel,Clothing Store,Asian Restaurant,Food Court,Gym
4,Bandar Menjalara,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station


In [36]:
KL_Areas_Sorted = Area_venues_sorted

In [37]:
KL_Areas_Sorted.to_csv('KLVenuesSorted.csv')


In [38]:
JB_onehot = pd.get_dummies(JB_venues[['Venue Category']], prefix="", prefix_sep="")
JB_onehot['Area'] = JB_venues['Area'] 
fixed_columns = [JB_onehot.columns[-1]] + list(JB_onehot.columns[:-1])
JB_onehot = JB_onehot[fixed_columns]

JB_onehot.head()

,Area,Afghan Restaurant,American Restaurant,Apres Ski Bar,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,...,Soup Place,Spa,Sports Club,Steakhouse,Supermarket,Thai Restaurant,Toll Plaza,Warehouse Store,Water Park,Wings Joint
0,Johor Bahru,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Johor Bahru,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Johor Bahru,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Johor Bahru,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Johor Bahru,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
JB_grouped = JB_onehot.groupby('Area').mean().reset_index()

In [40]:
num_top_venues = 5

for hood in JB_grouped['Area']:
    print("----"+hood+"----")
    temp = JB_grouped[JB_grouped['Area'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Tanjung Langsat----
                venue  freq
0    Asian Restaurant  0.21
1    Malay Restaurant  0.12
2  Chinese Restaurant  0.09
3     Thai Restaurant  0.06
4         Snack Place  0.06


----Bandar Dato' Onn----
                  venue  freq
0      Malay Restaurant  0.23
1                  Café  0.07
2    Seafood Restaurant  0.07
3     Convenience Store  0.05
4  Fast Food Restaurant  0.05


----Bandar Seri Alam----
                venue  freq
0    Asian Restaurant  0.21
1    Malay Restaurant  0.12
2  Chinese Restaurant  0.09
3     Thai Restaurant  0.06
4         Snack Place  0.06


----Bukit Indah----
              venue  freq
0  Malay Restaurant  0.14
1  Asian Restaurant  0.14
2   Harbor / Marina  0.14
3              Pool  0.14
4             Hotel  0.14


----Danga Bay----
              venue  freq
0  Malay Restaurant  0.18
1              Café  0.09
2          Boutique  0.06
3        Food Truck  0.06
4         Pet Store  0.05


----Desa Jaya----
                  venue  freq
0

In [41]:
#Based on the frequency distubution we can identenfy and correlate a list of the most common venues for each topic
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
AreaJB_venues_sorted = pd.DataFrame(columns=columns)
AreaJB_venues_sorted['Area'] = JB_grouped['Area']

for ind in np.arange(JB_grouped.shape[0]):
    AreaJB_venues_sorted.iloc[ind, 1:] = return_most_common_venues(JB_grouped.iloc[ind, :], num_top_venues)

AreaJB_venues_sorted.head()

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tanjung Langsat,Asian Restaurant,Malay Restaurant,Chinese Restaurant,Dessert Shop,Thai Restaurant,Snack Place,Coffee Shop,Convenience Store,Indian Restaurant,Golf Course
1,Bandar Dato' Onn,Malay Restaurant,Seafood Restaurant,Café,Convenience Store,Fast Food Restaurant,Hotel,Hookah Bar,Halal Restaurant,Hakka Restaurant,Gymnastics Gym
2,Bandar Seri Alam,Asian Restaurant,Malay Restaurant,Chinese Restaurant,Dessert Shop,Thai Restaurant,Snack Place,Coffee Shop,Convenience Store,Indian Restaurant,Golf Course
3,Bukit Indah,Hotel,Asian Restaurant,Burger Joint,Malay Restaurant,Harbor / Marina,Pool,Seafood Restaurant,Donut Shop,Coffee Shop,Convenience Store
4,Danga Bay,Malay Restaurant,Café,Boutique,Food Truck,Burger Joint,Steakhouse,Pet Store,Asian Restaurant,Bakery,Department Store


In [42]:
AreaJB_venues_sorted.to_csv('JBVenuesSorted.csv')

***

In [49]:
AreaJB_venues_sorted

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tanjung Langsat,Asian Restaurant,Malay Restaurant,Chinese Restaurant,Dessert Shop,Thai Restaurant,Snack Place,Coffee Shop,Convenience Store,Indian Restaurant,Golf Course
1,Bandar Dato' Onn,Malay Restaurant,Seafood Restaurant,Café,Convenience Store,Fast Food Restaurant,Hotel,Hookah Bar,Halal Restaurant,Hakka Restaurant,Gymnastics Gym
2,Bandar Seri Alam,Asian Restaurant,Malay Restaurant,Chinese Restaurant,Dessert Shop,Thai Restaurant,Snack Place,Coffee Shop,Convenience Store,Indian Restaurant,Golf Course
3,Bukit Indah,Hotel,Asian Restaurant,Burger Joint,Malay Restaurant,Harbor / Marina,Pool,Seafood Restaurant,Donut Shop,Coffee Shop,Convenience Store
4,Danga Bay,Malay Restaurant,Café,Boutique,Food Truck,Burger Joint,Steakhouse,Pet Store,Asian Restaurant,Bakery,Department Store
5,Desa Jaya,Malay Restaurant,Seafood Restaurant,Café,Convenience Store,Fast Food Restaurant,Hotel,Hookah Bar,Halal Restaurant,Hakka Restaurant,Gymnastics Gym
6,Ehsan Jaya,Malay Restaurant,Seafood Restaurant,Café,Convenience Store,Fast Food Restaurant,Hotel,Hookah Bar,Halal Restaurant,Hakka Restaurant,Gymnastics Gym
7,Gelang Patah,Breakfast Spot,Thai Restaurant,Asian Restaurant,Café,Malay Restaurant,Halal Restaurant,Indian Restaurant,Convenience Store,Burger Joint,Seafood Restaurant
8,Horizon Hills,Hotel,Asian Restaurant,Burger Joint,Malay Restaurant,Harbor / Marina,Pool,Seafood Restaurant,Donut Shop,Coffee Shop,Convenience Store
9,Johor Bahru,Malay Restaurant,Convenience Store,Café,Food Court,Snack Place,Paper / Office Supplies Store,Donut Shop,Pizza Place,Bookstore,Breakfast Spot


In [43]:
# set number of clusters
kclusters = 5

KL_grouped_clustering = KL_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(KL_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 1, 3, 4, 4, 1, 1, 1])

In [44]:
# add clustering labels and combining to add latitude/longitude for each Area
Area_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

KL_clusted = UsedKL


KL_clusted = KL_clusted.join(Area_venues_sorted.set_index('Area'), on='Area')

KL_clusted.head() # check the last columns!

,Postcode,District,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,52100,Kepong,Jinjang,3.1833,101.6584,3,Chinese Restaurant,Korean Restaurant,Café,Coffee Shop,Japanese Restaurant,Convenience Store,Malay Restaurant,Noodle House,Thai Restaurant,Shopping Mall
1,52100,Kepong,Taman Bukit Maluri,3.1833,101.6584,3,Chinese Restaurant,Korean Restaurant,Café,Coffee Shop,Japanese Restaurant,Convenience Store,Malay Restaurant,Noodle House,Thai Restaurant,Shopping Mall
2,51200,Segambut,Bandar Menjalara,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
3,51200,Segambut,Bukit Kiara,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
4,51200,Segambut,Bukit Tunku,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station


In [48]:
KL_clusted

,Postcode,District,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,52100,Kepong,Jinjang,3.1833,101.6584,3,Chinese Restaurant,Korean Restaurant,Café,Coffee Shop,Japanese Restaurant,Convenience Store,Malay Restaurant,Noodle House,Thai Restaurant,Shopping Mall
1,52100,Kepong,Taman Bukit Maluri,3.1833,101.6584,3,Chinese Restaurant,Korean Restaurant,Café,Coffee Shop,Japanese Restaurant,Convenience Store,Malay Restaurant,Noodle House,Thai Restaurant,Shopping Mall
2,51200,Segambut,Bandar Menjalara,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
3,51200,Segambut,Bukit Kiara,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
4,51200,Segambut,Bukit Tunku,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
5,51200,Segambut,Damansara,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
6,51200,Segambut,Damansara Town Centre,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
7,51200,Segambut,Jalan Duta,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
8,51200,Segambut,Kampung Kasipillay,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station
9,51200,Segambut,Kampung Sungai Penchala,3.1876,101.6720,3,Chinese Restaurant,Café,Asian Restaurant,Gym / Fitness Center,Burger Joint,Indian Restaurant,Malay Restaurant,Food Truck,Restaurant,Light Rail Station


In [45]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(KL_clusted['Latitude'], KL_clusted['Longitude'], KL_clusted['Area'], KL_clusted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
# set number of clusters
kclusters = 5

JB_grouped_clustering = JB_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(KL_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 1, 3, 4, 4, 1, 1, 1])

In [47]:
# add clustering labels and combining to add latitude/longitude for each Area
AreaJB_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

JB_clusted = UsedJB


JB_clusted = JB_clusted.join(AreaJB_venues_sorted.set_index('Area'), on='Area')

JB_clusted.head() # check the last columns!

ValueError: Length of values does not match length of index